## Getting data table for yesterday:

## Importing packages:

In [2]:
import requests #request the html page 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sqlalchemy as sqla

## Using BeautifulSoup parse yesterday's online tables:

In [6]:
url = 'https://www.worldometers.info/coronavirus/#countries' ###
scraping_page = requests.get(url)
scrapinghtml = scraping_page.text
scrapingsoup = BeautifulSoup(scrapinghtml,'lxml')
extract_table_today = scrapingsoup.find('table', id= 'main_table_countries_yesterday2')
extract_table_data_today = extract_table_today.find_all("tr")


## Storing all data in list:

In [31]:
deepthinker=[]
content = []
for i in range(9,len(extract_table_data_today)): 
        for j in range(len(extract_table_data_today[i].find_all("td",href = False))):
            content.append(extract_table_data_today[i].find_all("td",href = False)[j].text)
        deepthinker.append(content)
        content=[]

## Turning list into dataframe and match column names:

In [32]:
dataframe_today = pd.DataFrame(deepthinker)
%run ../code/dataframe_organizer.py
df = dataframe_organizer(dataframe_today)

## Adding yesterday's date as column to the dataframe:

In [33]:
from datetime import date, timedelta
yesterday = str(date.today() - timedelta(days=2))
yesterday = yesterday.replace("-",'')
df['date'] = [yesterday]*222

## Adding the dataframe to SQL database:

In [34]:
sqlite_file='../data/covid.sqlite'
covid_engine = sqla.create_engine('sqlite:///'+sqlite_file)
df.to_sql('date_table',con=covid_engine,if_exists='append')

In [35]:
pd.read_sql_query("select count(*) from date_table",covid_engine)

,count(*)
0,2220
